<a href="https://colab.research.google.com/github/Aunabil4602/Deep-Learning-on-tensorflow-datasets/blob/master/imdb_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import os
import numpy as np
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
import tensorflow_datasets as tfds

from keras.initializers import Constant


vocab_size = 20000
embedding_dim = 100
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [0]:
!unzip glove*.zip

In [0]:
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

num_words = min(vocab_size, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector



In [0]:
len(embeddings_index)

In [0]:
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)
train_data,test_data=imdb['train'],imdb['test']

training_sentences = []
training_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()).lower())
  training_labels.append(l.numpy())
  
testing_sentences = []
testing_labels = []

for s,l in test_data:
  testing_sentences.append(str(s.numpy()).lower())
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
print(len(training_sentences))
print(len(testing_sentences))

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index 
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type,padding='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,embeddings_initializer=Constant(embedding_matrix),input_length=max_length,trainable=False),
    #tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')                         
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [0]:
num_epochs=10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))